In [1]:
from typing import Optional, List, Dict
import os
import time

In [2]:
config = {
    'inputdir': '/eos/atlas/atlascerngroupdisk/phys-hdbs/diHiggs/combination/FullRun2Workspaces/original/20220204_HEFT',
    # change your output directory here
    'outdir': os.path.join(os.getcwd(), "outputs"),
    'scenarios': ['SM'] + [f'BM{i}' for i in range(1, 7 + 1)],
    'channels': ['bbyy', 'bbtautau'],
    'resonant_type': 'nonres',
    'file_expr': '<mass[F]>',
    'correlation_scheme': 'HEFT_nonres_v1.json',
    'task_option': 'HEFT_nonres_v1.yaml',
    'parallel': -1
}

In [3]:
def construct_commands(config:Dict):
    channels = ','.join(config['channels'])
    hhcomb_path = os.environ.get('hh_combination_fw_path', None)
    if hhcomb_path is None:
        raise RuntimeError("hh combination framework is not set up properly, please run setup.sh first")
    task_option_path = os.path.join(hhcomb_path, "configs", "task_options", config['task_option'])
    correlation_scheme_path = os.path.join(hhcomb_path, "configs", "correlation_schemes", config['correlation_scheme'])
    if not os.path.exists(task_option_path):
        raise FileNotFoundError(f"file \"{task_option_path}\" does not exist")
    if not os.path.exists(correlation_scheme_path):
        raise FileNotFoundError(f"file \"{correlation_scheme_path}\" does not exist")
    file_expr = config['file_expr']
    commands = {'process_channels': [], 'combine_ws': []}
    for scenario in config['scenarios']:
        input_path = os.path.join(config['inputdir'], scenario)
        output_path = os.path.join(config['outdir'], scenario)
        # process channels
        command_channel = f"HHComb process_channels -i {input_path} -c \"{channels}\" -r {config['resonant_type']} " +\
                          f"-o {output_path} --file_expr \"{file_expr}\" --config {task_option_path} " +\
                          f"--parallel {config['parallel']} --unblind"
        commands['process_channels'].append(command_channel)
        # combination
        command_comb = f"HHComb combine_ws -i {output_path} -s {correlation_scheme_path} -c \"{channels}\" " +\
                       f"-r {config['resonant_type']} --file_expr \"{file_expr}\" --config {task_option_path} " +\
                       f"--parallel {config['parallel']} --unblind"
        commands['combine_ws'].append(command_comb)
    for command_type in commands:
        # parallelization
        commands[command_type] = ' & '.join(commands[command_type])
    return commands

In [4]:
commands = construct_commands(config)

In [5]:
print("# commands for processing individual channels")
print()
print(commands['process_channels'])
print()
print("# commands for combination")
print()
print(commands['combine_ws'])

# commands for processing individual channels

HHComb process_channels -i /eos/atlas/atlascerngroupdisk/phys-hdbs/diHiggs/combination/FullRun2Workspaces/original/20220204_HEFT/SM -c "bbyy,bbtautau" -r nonres -o /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/tutorials/HEFT_combination/outputs/SM --file_expr "<mass[F]>" --config /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/configs/task_options/HEFT_nonres_v1.yaml --parallel -1 --unblind & HHComb process_channels -i /eos/atlas/atlascerngroupdisk/phys-hdbs/diHiggs/combination/FullRun2Workspaces/original/20220204_HEFT/BM1 -c "bbyy,bbtautau" -r nonres -o /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/tutorials/HEFT_combination/outputs/BM1 --file_expr "<mass[F]>" --config /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/configs/task_options/HEFT_nonres_v1.yaml --parallel -1 --unblind & HHComb process_channels -i /eos/atlas/atlascerngroupdisk/phys-hdbs/diHiggs/combination/FullRun2Workspaces/original/

In [ ]:
!{commands['process_channels']}

In [ ]:
!{commands['combine_ws']}


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.t

INFO: Writing combination log into /afs/cern.ch/work/c/chlcheng/Repository/hh_combination_fw/tutorials/HEFT_combination/outputs/BM1/combined/nonres/A-bbtautau_bbyy-fullcorr/0.log


## Plot limit results

In [45]:
import json
import numpy as np
import pandas as pd

In [46]:
limit_data = {}
for channel in config['channels'] + ['combined']:
    limit_data[channel] = {}
    for scenario in config['scenarios']:
        outdir = os.path.join(config['outdir'], scenario, 'limits', config['resonant_type'], channel)
        if channel != 'combined':
            limit_path = os.path.join(outdir, "limits.json")
        else:
            limit_path = os.path.join(outdir, "A-{}-fullcorr".format("_".join(sorted(config['channels']))), "limits.json")
        if not os.path.exists(limit_path):
            raise FileNotFoundError(f"file \"{limit_path}\" not found")
        limit_data[channel][scenario] = {k:v[0] for k,v in json.load(open(limit_path)).items()}
    limit_data[channel] = pd.DataFrame(limit_data[channel]).transpose()

In [47]:
limit_data['bbyy']

,mass,0,2,1,-1,-2,obs,inj
SM,0.0,0.171222,0.379666,0.253113,0.123375,0.091899,0.127700,0.0
BM1,0.0,0.292286,0.630573,0.428016,0.210608,0.156877,0.189232,0.0
BM2,0.0,0.266603,0.580714,0.391620,0.192102,0.143093,0.182984,0.0
BM3,0.0,0.162769,0.365492,0.241652,0.117284,0.087362,0.109703,0.0
BM4,0.0,0.155404,0.346426,0.230125,0.111977,0.083409,0.111165,0.0
BM5,0.0,0.137545,0.308707,0.204128,0.099109,0.073824,0.097866,0.0
BM6,0.0,0.188601,0.418513,0.278912,0.135898,0.101227,0.134407,0.0
BM7,0.0,0.125969,0.282098,0.186804,0.090768,0.067611,0.088828,0.0


In [48]:
limit_data['bbtautau']

,mass,0,2,1,-1,-2,obs,inj
SM,0.0,0.108757,0.218531,0.155066,0.078366,0.058373,0.128198,0.0
BM1,0.0,0.148557,0.307472,0.213997,0.107044,0.079734,0.193217,0.0
BM2,0.0,0.161797,0.331189,0.232190,0.116584,0.086841,0.199791,0.0
BM3,0.0,0.061997,0.133849,0.090720,0.044673,0.033276,0.082125,0.0
BM4,0.0,0.074618,0.151183,0.106754,0.053766,0.040049,0.093310,0.0
BM5,0.0,0.060842,0.127198,0.087907,0.043840,0.032655,0.077635,0.0
BM6,0.0,0.100606,0.203652,0.143859,0.072493,0.053998,0.124165,0.0
BM7,0.0,0.050617,0.103298,0.072599,0.036472,0.027167,0.065253,0.0


In [49]:
limit_data['combined']

,mass,0,2,1,-1,-2,obs,inj
SM,0.0,0.088149,0.176753,0.125584,0.063517,0.047312,0.086830,0.0
BM1,0.0,0.128604,0.260998,0.183997,0.092666,0.069025,0.134037,0.0
BM2,0.0,0.133545,0.269903,0.190756,0.096226,0.071677,0.133587,0.0
BM3,0.0,0.056163,0.118825,0.081505,0.040469,0.030144,0.062550,0.0
BM4,0.0,0.064895,0.130526,0.092577,0.046760,0.034831,0.068577,0.0
BM5,0.0,0.053716,0.110651,0.077229,0.038705,0.028831,0.058046,0.0
BM6,0.0,0.085543,0.172059,0.122014,0.061639,0.045913,0.088284,0.0
BM7,0.0,0.045475,0.091944,0.065002,0.032767,0.024407,0.050248,0.0


In [54]:
for channel in limit_data:
    output_path = os.path.join(config['outdir'], f"BM_limits_{channel}.json")
    json.dump(limit_data['combined'].to_dict('index'), open(output_path, "w"), indent=2)